# Install necessary packages

In [21]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Part 1: Introduction to Classification & Evaluation

**Objective:** Load the synthetic health data, train a Logistic Regression model, and evaluate its performance.

## 1. Setup

Import necessary libraries.

In [22]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer

## 2. Data Loading

Implement the `load_data` function to read the dataset.

In [23]:
def load_data(file_path):
    """
    Load the synthetic health data from a CSV file.
    
    Args:
        file_path: Path to the CSV file
        
    Returns:
        DataFrame containing the data
    """
    data = pd.read_csv(file_path)
    return data  

## 3. Data Preparation

Implement `prepare_data_part1` to select features, split data, and handle missing values.

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
def prepare_data_part1(df, test_size=0.2, random_state=42):
    """
    Prepare data for modeling: select features, split into train/test sets, handle missing values.
    
    Args:
        df: Input DataFrame
        test_size: Proportion of data for testing
        random_state: Random seed for reproducibility
        
    Returns:
        X_train, X_test, y_train, y_test
    """
    X = df[['age', 'systolic_bp', 'diastolic_bp', 'glucose_level', 'bmi']]
    y = df['disease_outcome']

    imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')
    X = imputer.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

## 4. Model Training

Implement `train_logistic_regression`.

In [25]:
from sklearn.linear_model import LogisticRegression
def train_logistic_regression(X_train, y_train):
    """
    Train a logistic regression model.
    
    Args:
        X_train: Training features
        y_train: Training target
        
    Returns:
        Trained logistic regression model
    """
    logreg = LogisticRegression(random_state=42)
    return logreg.fit(X_train, y_train)  

## 5. Model Evaluation

Implement `calculate_evaluation_metrics` to assess the model performance.

In [26]:
from sklearn.metrics import classification_report
def calculate_evaluation_metrics(model, X_test, y_test):
    """
    Calculate classification evaluation metrics.
    
    Args:
        model: Trained model
        X_test: Test features
        y_test: Test target
        
    Returns:
        Dictionary containing accuracy, precision, recall, f1, auc, and confusion_matrix
    """

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary')
    recall = recall_score(y_test, y_pred, average='binary')
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    cnf_matrix = confusion_matrix(y_test, y_pred)
    cm = cnf_matrix.tolist()

    return {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'auc': auc,
    'confusion_matrix': cm
    }

## 6. Save Results

Save the calculated metrics to a text file.

In [27]:
import os 
import json

def save_results(metrics: dict):
    os.makedirs('results', exist_ok=True)
    formatted_metrics = [f"{key}: {value}" for key, value in metrics.items()]

    output_file = 'results/results_part1.txt'
    with open('results/results_part1.txt', 'w') as f:
        json.dump(metrics, f, indent=2)
    print(f"\nMetrics saved to {output_file}")

## 8. Interpret Results

Implement a function to analyze the model performance on imbalanced data.


In [28]:
def interpret_results(metrics):
    """
    Analyze model performance on imbalanced data.
    
    Args:
        metrics: Dictionary containing evaluation metrics
        
    Returns:
        Dictionary with keys:
        - 'best_metric': Name of the metric that performed best
        - 'worst_metric': Name of the metric that performed worst
        - 'imbalance_impact_score': A score from 0-1 indicating how much
          the class imbalance affected results (0=no impact, 1=severe impact)
    """
    best_metric = max(metrics, key=lambda k: float(metrics[k]))
    worst_metric = min(metrics, key=lambda k: float(metrics[k]))

    imbalance_sensitive = [float(metrics[k]) for k in ('f1', 'recall') if k in metrics]
    accuracy = float(metrics.get('accuracy', 0))
    gaps = [max(0, accuracy - val) for val in imbalance_sensitive]

    imbalance_impact_score = float(np.clip(np.mean(gaps), 0, 1)) if gaps else 0.0

    return {
        'best_metric': best_metric,
        'worst_metric': worst_metric,
        'imbalance_impact_score': imbalance_impact_score
    }

## 7. Main Execution

Run the complete workflow.

In [29]:
# Main execution
if __name__ == "__main__":
    # 1. Load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # 2. Prepare data
    X_train, X_test, y_train, y_test = prepare_data_part1(df)
    
    # 3. Train model
    model = train_logistic_regression(X_train, y_train)
    
    # 4. Evaluate model
    metrics = calculate_evaluation_metrics(model, X_test, y_test)
    
    # 5. Print metrics
    for metric, value in metrics.items():
        if metric != 'confusion_matrix':
            print(f"{metric}: {value:.4f}")
    
    # 6. Save results
    save_results(metrics)
    
    # 7. Interpret results
    scalar_metrics = {k: v for k, v in metrics.items() if k != 'confusion_matrix'}
    interpretation = interpret_results(scalar_metrics)
    print("\nResults Interpretation:")
    for key, value in interpretation.items():
        print(f"{key}: {value}")

accuracy: 0.9168
precision: 0.6615
recall: 0.3007
f1_score: 0.4135
auc: 0.6420

Metrics saved to results/results_part1.txt

Results Interpretation:
best_metric: accuracy
worst_metric: recall
imbalance_impact_score: 0.6160810540073842
